This notebook can be run on the Copernicus Dataspace Jupyterhub but running the following package installation cell first

**Note** You should select on of the kernels with GDAL installed, eg. "Geo science"

In [1]:
try:
    import eomaji
except ModuleNotFoundError:
    !pip install eomaji@git+https://github.com/DHI/EOMAJI-OpenEO-toolbox.git

# Prepare data for PyDMS and ET Flows

In [2]:
import openeo
from eomaji.utils import draw_utils, date_selector
from eomaji.utils.general_utils import dump_area_date_info
from eomaji.workflows import prepare_data_cubes

## 1. Select Area of Interest
Draw a polygon on the map, for the area you want to process

In [3]:
map, bboxs = draw_utils.draw_aoi()
map

Map(center=[40, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

## 2. Select Date from Available Dates for Sentinel-3
Search for days where sentinel 3 images are available for the given AOI. 

**Note**: max_cloud_cover refers to the full tile’s coverage, not just your AOI, so results may vary.

In [4]:
# Define search parameters
start_date = "2023-05-01"
end_date = "2023-07-30"
bbox = bboxs[-1] # If you didn't draw a polygon, please insert a bbox here in the form of [minx, miny, maxx, maxy]
max_cloud_cover = 10  # Filter out high-cloud-coverage scenes

# Search for available Sentinel-3 imagery
date_selection = date_selector.get_available_dates(
    start_date=start_date,
    end_date=end_date,
    bbox=bbox,
    max_cloud_cover=max_cloud_cover
)

Dropdown(description='Pick a Date:', options=(('2023-05-12', datetime.date(2023, 5, 12)), ('2023-05-31', datet…

## 3. Connect to OpenEO Backend

In [5]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

2025-06-19 13:26:55,490 [INFO] Loaded openEO client config from sources: []
2025-06-19 13:26:56,193 [INFO] Found OIDC providers: ['CDSE']
2025-06-19 13:26:56,195 [INFO] No OIDC provider given, but only one available: 'CDSE'. Using that one.
2025-06-19 13:26:56,567 [INFO] Using default client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e' from OIDC provider 'CDSE' info.
2025-06-19 13:26:56,571 [INFO] Found refresh token: trying refresh token based authentication.
2025-06-19 13:26:56,574 [INFO] Doing 'refresh_token' token request 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' with post data fields ['grant_type', 'client_id', 'refresh_token'] (client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e')
2025-06-19 13:26:56,958 [INFO] Obtained tokens: ['access_token', 'id_token', 'refresh_token']
2025-06-19 13:26:56,960 [INFO] Storing refresh token for issuer 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE' (client 'sh-b1c3a958-52d4-40fe-a333-153

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

## 4. Download Sentinel 2 and Sentinel 3 data for the specified AOI and date

In [6]:
s2_path, s3_path, worldcover_path, dem_s2_path, dem_s3_path, acq_time = prepare_data_cubes.prepare_data_cubes(
    connection=connection,
    bbox=bbox,
    date=date_selection.value,
    sentinel2_search_range = 3,
    out_dir = "./data",
)

0:00:00 Job 'j-25061913271648888b72eca5e23908a0': send 'start'
0:00:19 Job 'j-25061913271648888b72eca5e23908a0': created (progress 0%)
0:00:25 Job 'j-25061913271648888b72eca5e23908a0': created (progress 0%)
0:00:31 Job 'j-25061913271648888b72eca5e23908a0': created (progress 0%)
0:00:39 Job 'j-25061913271648888b72eca5e23908a0': created (progress 0%)
0:00:49 Job 'j-25061913271648888b72eca5e23908a0': created (progress 0%)
0:01:02 Job 'j-25061913271648888b72eca5e23908a0': running (progress N/A)
0:01:17 Job 'j-25061913271648888b72eca5e23908a0': running (progress N/A)
0:01:36 Job 'j-25061913271648888b72eca5e23908a0': running (progress N/A)
0:02:00 Job 'j-25061913271648888b72eca5e23908a0': running (progress N/A)
0:02:30 Job 'j-25061913271648888b72eca5e23908a0': running (progress N/A)
0:03:08 Job 'j-25061913271648888b72eca5e23908a0': running (progress N/A)
0:03:55 Job 'j-25061913271648888b72eca5e23908a0': running (progress N/A)
0:04:53 Job 'j-25061913271648888b72eca5e23908a0': running (progres

2025-06-19 13:35:11,360 [INFO] Downloading Job result asset 'openEO.nc' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-25061913271648888b72eca5e23908a0/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/6927327f9ed48666e3766286b3283c6f/openEO.nc?expires=1750944911 to data/20230531_00e8dda8/s2_data.nc
/usr/local/lib/python3.10/dist-packages/openeo/rest/connection.py:1171: UserWarning: Property filtering with unsupported properties according to collection/STAC metadata: {'timeliness', 'orbitDirection'} (supported: dict_keys(['bands', 'eo:bands', 'raster:bands'])).
  return DataCube.load_collection(


0:00:00 Job 'j-250619133616487c8d324cb92958b5df': send 'start'
0:00:13 Job 'j-250619133616487c8d324cb92958b5df': created (progress 0%)
0:00:18 Job 'j-250619133616487c8d324cb92958b5df': created (progress 0%)
0:00:25 Job 'j-250619133616487c8d324cb92958b5df': created (progress 0%)
0:00:33 Job 'j-250619133616487c8d324cb92958b5df': created (progress 0%)
0:00:43 Job 'j-250619133616487c8d324cb92958b5df': created (progress 0%)
0:00:56 Job 'j-250619133616487c8d324cb92958b5df': running (progress N/A)
0:01:11 Job 'j-250619133616487c8d324cb92958b5df': running (progress N/A)
0:01:31 Job 'j-250619133616487c8d324cb92958b5df': running (progress N/A)
0:01:54 Job 'j-250619133616487c8d324cb92958b5df': running (progress N/A)
0:02:24 Job 'j-250619133616487c8d324cb92958b5df': finished (progress 100%)


2025-06-19 13:38:42,331 [INFO] Downloading Job result asset 'openEO.nc' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-250619133616487c8d324cb92958b5df/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/d7823b31ec515b61797076db90baf253/openEO.nc?expires=1750945122 to data/20230531_00e8dda8/s3_data.nc


0:00:00 Job 'j-2506191338434917915c102304cf6bd1': send 'start'
0:00:13 Job 'j-2506191338434917915c102304cf6bd1': created (progress 0%)
0:00:18 Job 'j-2506191338434917915c102304cf6bd1': created (progress 0%)
0:00:24 Job 'j-2506191338434917915c102304cf6bd1': running (progress N/A)
0:00:32 Job 'j-2506191338434917915c102304cf6bd1': running (progress N/A)
0:00:42 Job 'j-2506191338434917915c102304cf6bd1': running (progress N/A)
0:00:55 Job 'j-2506191338434917915c102304cf6bd1': running (progress N/A)
0:01:10 Job 'j-2506191338434917915c102304cf6bd1': running (progress N/A)
0:01:29 Job 'j-2506191338434917915c102304cf6bd1': running (progress N/A)
0:01:53 Job 'j-2506191338434917915c102304cf6bd1': running (progress N/A)
0:02:23 Job 'j-2506191338434917915c102304cf6bd1': running (progress N/A)
0:03:01 Job 'j-2506191338434917915c102304cf6bd1': finished (progress 100%)


2025-06-19 13:41:44,939 [INFO] Downloading Job result asset 'openEO_2011-01-27Z.tif' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-2506191338434917915c102304cf6bd1/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/3d1baef8599213f343ad78294575db99/openEO_2011-01-27Z.tif?expires=1750945304 to data/20230531_00e8dda8/20230531_ELEV.tif


0:00:00 Job 'j-25061913415549c6bb31c0a3fee6dd44': send 'start'
0:00:13 Job 'j-25061913415549c6bb31c0a3fee6dd44': queued (progress 0%)
0:00:18 Job 'j-25061913415549c6bb31c0a3fee6dd44': queued (progress 0%)
0:00:25 Job 'j-25061913415549c6bb31c0a3fee6dd44': queued (progress 0%)
0:00:33 Job 'j-25061913415549c6bb31c0a3fee6dd44': queued (progress 0%)
0:00:43 Job 'j-25061913415549c6bb31c0a3fee6dd44': queued (progress 0%)
0:00:55 Job 'j-25061913415549c6bb31c0a3fee6dd44': queued (progress 0%)
0:01:10 Job 'j-25061913415549c6bb31c0a3fee6dd44': queued (progress 0%)
0:01:30 Job 'j-25061913415549c6bb31c0a3fee6dd44': running (progress N/A)
0:01:54 Job 'j-25061913415549c6bb31c0a3fee6dd44': running (progress N/A)
0:02:24 Job 'j-25061913415549c6bb31c0a3fee6dd44': finished (progress 100%)


2025-06-19 13:44:19,564 [INFO] Downloading Job result asset 'openEO_2011-01-27Z.tif' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-25061913415549c6bb31c0a3fee6dd44/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/5297db99159a7a0fc080681bde03e581/openEO_2011-01-27Z.tif?expires=1750945459 to data/20230531_00e8dda8/meteo_dem.tif


0:00:00 Job 'j-2506191344204c43a2b3a0a1ed3e1371': send 'start'
0:00:12 Job 'j-2506191344204c43a2b3a0a1ed3e1371': created (progress 0%)
0:00:17 Job 'j-2506191344204c43a2b3a0a1ed3e1371': created (progress 0%)
0:00:24 Job 'j-2506191344204c43a2b3a0a1ed3e1371': created (progress 0%)
0:00:32 Job 'j-2506191344204c43a2b3a0a1ed3e1371': created (progress 0%)
0:00:42 Job 'j-2506191344204c43a2b3a0a1ed3e1371': created (progress 0%)
0:00:54 Job 'j-2506191344204c43a2b3a0a1ed3e1371': running (progress N/A)
0:01:10 Job 'j-2506191344204c43a2b3a0a1ed3e1371': running (progress N/A)
0:01:29 Job 'j-2506191344204c43a2b3a0a1ed3e1371': running (progress N/A)
0:01:53 Job 'j-2506191344204c43a2b3a0a1ed3e1371': finished (progress 100%)


2025-06-19 13:46:14,516 [INFO] Downloading Job result asset 'openEO_2021-01-01Z.tif' from https://openeo.dataspace.copernicus.eu/openeo/1.2/jobs/j-2506191344204c43a2b3a0a1ed3e1371/results/assets/ZTlhMGJlNzktY2QxYy00YzFmLWI0OTgtZjJiZTNjZTYwN2Mx/c97da47ad84a62aa8fe08b6398ed672c/openEO_2021-01-01Z.tif?expires=1750945574 to data/20230531_00e8dda8/WordlCover2021.tif
2025-06-19 13:46:15,447 [INFO] Data cubes prepared and saved.


### Store AOI and Date for use in other notebooks    

In [7]:
dump_area_date_info(
    date = date_selection.value, 
    bbox = bbox, 
    out_dir = "./data"
)